In [ ]:
%pip install -U "transformers>=4.30" "accelerate>=0.26" torch datasets scikit-learn numpy pandas

In [ ]:
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict


In [ ]:
name = "cointegrated/rubert-tiny2"
model = AutoModelForSequenceClassification.from_pretrained(name,
        id2label = { 0: "0", 1: "1", 2: "2", 3: "3"},
        label2id = { "0": 0, "1": 1, "2": 2, "3": 3},           
    )
tok = AutoTokenizer.from_pretrained(name, model_max_length=512)
train = pd.read_csv("../data/output/train.csv")
train = train.rename(columns={"type": "labels"})
test = pd.read_csv("../data/output/test.csv")
test = train.rename(columns={"type": "labels"})
data = DatasetDict({
    "train":  Dataset.from_pandas(train[["text","labels"]]),
    "test": Dataset.from_pandas(test[["text","labels"]])
})

In [ ]:
def preprocessing(raw):
    batch = tok (raw["text"],
        truncation=True,
        max_length=512,
        padding=False 
    )
    return batch

In [ ]:
tokenized_dataset = data.map(preprocessing, batched = True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tok)

In [ ]:
import transformers
print("Transformers version:", transformers.__version__)

In [ ]:
args = TrainingArguments(
    output_dir = "../data/output",
    eval_strategy = "epoch",
    save_strategy="epoch",
    logging_strategy="steps", 
    logging_steps=20, 
    logging_first_step = True,
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tok,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

In [ ]:
preds = trainer.predict(tokenized_dataset["test"])
pred_labels = preds.predictions.argmax(axis = -1)
pred_labels